In [4]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import GPTNeoXForCausalLM
from sklearn.decomposition import PCA
from matplotlib.cm import get_cmap

# Define constants
experiment = "train_seq_just_DnD"
MODEL_NAME = "EleutherAI/pythia-70m"
MODEL_FOLDER = "/workspace/slice-monorepo/sub_validations/cl_scaling/experiments/train_seq_just_DnD/models"
OUTPUT_FOLDER = os.path.join(experiment, "mad_analysis_output")
PCA_FOLDER = os.path.join(OUTPUT_FOLDER, 'pca')
os.makedirs(PCA_FOLDER, exist_ok=True)

# Function to calculate MAD
def calculate_mad(weights1, weights2):
    return np.mean(np.abs(weights1 - weights2))

# Function to perform PCA and save the plot
def plot_pca(weights1, weights2, layer_name, batch_num):
    try:
        if weights1.ndim < 2:
            weights1 = weights1.reshape(-1, 1)
            weights2 = weights2.reshape(-1, 1)
        pca = PCA(n_components=2)
        weights = np.concatenate([weights1, weights2], axis=0)
        pca_result = pca.fit_transform(weights)
        plt.figure(figsize=(10, 5))
        plt.scatter(pca_result[:len(weights1), 0], pca_result[:len(weights1), 1], alpha=0.5, label='Pre-trained')
        plt.scatter(pca_result[len(weights1):, 0], pca_result[len(weights1):, 1], alpha=0.5, label='Fine-tuned')
        plt.title(f'PCA of Weights: {layer_name} - Batch {batch_num}')
        plt.xlabel('PCA Component 1')
        plt.ylabel('PCA Component 2')
        plot_file = os.path.join(PCA_FOLDER, f'{layer_name.replace(".", "_")}_batch_{batch_num}_pca.png')
        plt.savefig(plot_file)
        plt.close()
    except Exception as e:
        print(f"Skipping PCA for {layer_name} due to error: {e}")

# Load models
def load_models(saved_model_path):
    model_pretrained = GPTNeoXForCausalLM.from_pretrained(MODEL_NAME).to('cuda')
    model_fine_tuned = GPTNeoXForCausalLM.from_pretrained(MODEL_NAME).to('cuda')
    model_fine_tuned.load_state_dict(torch.load(saved_model_path))
    return model_pretrained, model_fine_tuned

# Function to analyze models
def analyze_models(model_folder):
    model_pretrained = GPTNeoXForCausalLM.from_pretrained(MODEL_NAME).to('cuda')
    layer_names = [name for name, _ in model_pretrained.named_parameters() if "weight" in name]
    mad_values = []

    csv_mad_path = os.path.join(OUTPUT_FOLDER, 'mad_values.csv')
    cmap = get_cmap('viridis')

    if os.path.exists(csv_mad_path):
        df_mad = pd.read_csv(csv_mad_path)
        print(f'Loaded existing MAD values from {csv_mad_path}')
        file_creation_times = df_mad['Batch'].unique()
        norm = plt.Normalize(min(file_creation_times), max(file_creation_times))
    else:
        model_files = [f for f in os.listdir(model_folder) if f.startswith("model_epoch") and f.endswith(".pt") and not f.startswith("final_model")]
        model_files = sorted(model_files, key=lambda x: os.path.getctime(os.path.join(model_folder, x)))
        file_creation_times = [os.path.getctime(os.path.join(model_folder, f)) for f in model_files]
        norm = plt.Normalize(min(file_creation_times), max(file_creation_times))

        for file_name in model_files:
            model_path = os.path.join(model_folder, file_name)
            creation_time = os.path.getctime(model_path)
            model_fine_tuned = GPTNeoXForCausalLM.from_pretrained(MODEL_NAME).to('cuda')
            model_fine_tuned.load_state_dict(torch.load(model_path))

            # Calculate MAD for all layers
            for layer_name in layer_names:
                weights_pretrained = model_pretrained.state_dict()[layer_name].cpu().numpy()
                weights_fine_tuned = model_fine_tuned.state_dict()[layer_name].cpu().numpy()
                mad = calculate_mad(weights_pretrained.flatten(), weights_fine_tuned.flatten())
                mad_values.append({'Layer': layer_name, 'MAD': mad, 'Model': file_name, 'Batch': creation_time})
        
        # Save MAD values to CSV
        df_mad = pd.DataFrame(mad_values)
        df_mad.to_csv(csv_mad_path, index=False)
        print(f'MAD values saved to {csv_mad_path}')

    # Plot and save the MAD scatter plot
    plt.figure(figsize=(18, 6))
    unique_times = sorted(df_mad['Batch'].unique())
    selected_times = [unique_times[0], unique_times[len(unique_times)//3], unique_times[2*len(unique_times)//3], unique_times[-1]]
    handles = []
    labels = []
    for creation_time in unique_times:
        time_df = df_mad[df_mad['Batch'] == creation_time]
        color = cmap(norm(creation_time))
        scatter = plt.scatter(range(len(time_df)), time_df['MAD'], color=color, alpha=0.6)
        if creation_time in selected_times:
            handles.append(scatter)
            labels.append(f'Time {creation_time}')
    plt.xlabel('Layer Index')
    plt.ylabel('MAD')
    plt.title('MAD for Each Layer by Creation Time')
    plt.legend(handles, labels)
    plt.xticks(range(0, len(layer_names), 20))  # Adjusting x-ticks to show every 20th layer index
    plt.tight_layout()
    plot_mad_path = os.path.join(OUTPUT_FOLDER, 'mad_plot.png')
    plt.savefig(plot_mad_path)
    plt.close()
    print(f'MAD plot saved to {plot_mad_path}')

def main():
    # Analyze the models for MAD
    analyze_models(MODEL_FOLDER)

if __name__ == "__main__":
    main()


/tmp/ipykernel_16349/157268334.py:57: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = get_cmap('viridis')


MAD values saved to train_seq_just_DnD/mad_analysis_output/mad_values.csv
MAD plot saved to train_seq_just_DnD/mad_analysis_output/mad_plot.png
